# Using data from first project which used linear regression and using random forest classifier model to compare recommendations for new zip codes for Starbucks locations

In [1]:
from sklearn import tree
import pickle
import numpy as np
from pyzipcode import ZipCodeDatabase
import zipfile
import gmaps
import gmaps.datasets
import os
import pandas as pd
import requests
import json
from census import Census
import matplotlib.pyplot as plt
import requests
import json
from pygeocoder import Geocoder, GeocoderError
import scipy.stats as stats

# Census API Key
from config import census_key, gkey
c = Census(census_key, year=2013)
gmaps.configure(api_key=gkey)

#url for google geocode api
basegeo_url = "https://maps.googleapis.com/maps/api/geocode/json"

In [2]:
#Read list of zipcodes with census data
df = pd.read_csv(os.path.join(".", "Resources", "SB_Census_Data_forml.csv"))
df.head()

,Zipcode,Median Age,Household Income,Per Capita Income,Employment rate,Has Starbucks
0,601,36.6,12041,7380,0.308835,N
1,602,38.6,15663,8463,0.381047,N
2,603,38.9,15485,9176,0.314867,N
3,606,37.3,15019,6383,0.306630,N
4,610,39.2,16707,7892,0.321893,N


In [3]:
#Prepare target data
target = df["Has Starbucks"]
target_names = ["N", "Y"]

In [4]:
#Prepare factor data
data = df.drop(["Has Starbucks", "Zipcode"], axis=1)
feature_names = data.columns
data.head()

,Median Age,Household Income,Per Capita Income,Employment rate
0,36.6,12041,7380,0.308835
1,38.6,15663,8463,0.381047
2,38.9,15485,9176,0.314867
3,37.3,15019,6383,0.306630
4,39.2,16707,7892,0.321893


In [5]:
#Split data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [6]:
# Import Random Forest Classifier model and run
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_test, y_test)

0.8448168063023633

In [7]:
# See importance of each of the features
sorted(zip(rf.feature_importances_, feature_names), reverse=True)

[(0.2849270966720877, 'Per Capita Income'),
 (0.2537648479757056, 'Median Age'),
 (0.23541959033557214, 'Employment rate'),
 (0.22588846501663457, 'Household Income')]

In [8]:
# Save model for future use
rfPickle = open('rfpickle_file', 'wb') 

pickle.dump(rf, rfPickle)


# load the model from disk
#loaded_model = pickle.load(open('rfpickle_file', 'rb'))
#result = loaded_model.predict(X_test) 

In [9]:
# Split zip codes into with Starbucks and without
# Remove zip codes and save in separate dataframe
# Predict probability of zip codes where there is no Starbucks
nodf=df.loc[df["Has Starbucks"]=="N"]
yesdf=df.loc[df['Has Starbucks']=='Y']
zips = nodf["Zipcode"]
nodf = nodf.drop(["Has Starbucks", "Zipcode"], axis=1)
ynew = rf.predict_proba(nodf)

print(ynew)

[[0.99  0.01 ]
 [0.995 0.005]
 [1.    0.   ]
 ...
 [0.49  0.51 ]
 [0.985 0.015]
 [0.98  0.02 ]]


In [10]:
# Create dataframe with list of zipcodes with no Starbucks and reset index
zips = pd.DataFrame(zips)
zips = zips.reset_index(drop=True)


In [11]:
# Create dataframe with prediction data
output = pd.DataFrame(ynew)


In [12]:
# Merge zip codes with prediction data
df_out = pd.merge(output, zips, left_index=True, right_index=True)


In [13]:
# Sort zip codes by least probability of no Starbucks
recommended = df_out.sort_values(by=[0])
recommended.head()

,0,1,Zipcode
465,0.105,0.895,2127
23814,0.115,0.885,82636
501,0.120,0.880,2210
647,0.125,0.875,2802
5782,0.125,0.875,21793


In [14]:
#Create a list with all zipcodes within 2 miles of zipcodes with a Starbucks.

zcdb = ZipCodeDatabase()
rec_l = []

for zipcode in yesdf.Zipcode:
    try:
        rec_l.append([z.zip for z in zcdb.get_zipcodes_around_radius(zipcode, 2)])
    except:
        print("Error")

In [15]:
# Flatten list
flat_list_str = [item for sublist in rec_l for item in sublist]

In [16]:
# Add column indicating nearby store to recommendation dataframe
recommended['Another_Store_Present'] = recommended.Zipcode.isin(flat_list_str).astype(int)

In [17]:
# Show how many zipcodes have nearby stores
recommended['Another_Store_Present'].value_counts()

0    25262
1     1357
Name: Another_Store_Present, dtype: int64

In [18]:
# Show dataframe with all columns
no_store_near_df = recommended.loc[recommended["Another_Store_Present"]==0]
no_store_near_df.head()

,0,1,Zipcode,Another_Store_Present
465,0.105,0.895,2127,0
23814,0.115,0.885,82636,0
501,0.120,0.880,2210,0
647,0.125,0.875,2802,0
19972,0.135,0.865,68136,0


In [19]:
# Export csv file with dataframe
no_store_near_df.to_csv(os.path.join(".", "Resources", "no_store_near_df.csv"))

In [20]:
# Import recommendations from first project
project1_df = pd.read_csv(os.path.join(".", "Resources", "Recommended_Zipcodes.csv"))
project1_df.head()

,Unnamed: 0,Unnamed: 0.1,Zipcode,Population,Median Age,Household Income,Per Capita Income,Poverty Count,Employment Labor Force,Poverty Rate,...,State/Province,Country,Postcode,Phone Number,Timezone,Longitude,Latitude,Employment_PCT,Recommended,Another_Store_Present
0,0,0,8518,5217.0,41.5,74286.0,33963.0,170.0,3220.0,3.258578,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.721296,Y,0
1,1,1,8520,27468.0,37.4,90293.0,37175.0,1834.0,16187.0,6.676860,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58.930392,Y,0
2,3,3,8527,54867.0,42.2,88588.0,37021.0,2191.0,29204.0,3.993293,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.226894,Y,0
3,4,4,8528,245.0,48.5,58676.0,49117.0,0.0,130.0,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.061224,Y,0
4,5,5,8530,7568.0,43.8,87396.0,46371.0,249.0,4058.0,3.290169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53.620507,Y,0


In [21]:
# Filter results for above 50 percent probability
above50pct=no_store_near_df.loc[no_store_near_df[0]<=0.5]

In [22]:
# Count how many zip codes were above 50 percent
len(above50pct.index)

276

In [23]:
above10pct=no_store_near_df.loc[no_store_near_df[0]<=0.9]
len(above10pct.index)

5183

In [24]:
# Merge above 50 pct with first project recommendations
common_zip_codes = pd.merge(above50pct, project1_df, how = 'inner', on = 'Zipcode')

In [25]:
# Eliminate duplicate zip codes and count common zip codes
unique = common_zip_codes.groupby(['Zipcode']).all()
len(unique.index)

149

In [26]:
# Output unique common zip codes to csv file
unique.to_csv(os.path.join(".", "Resources", "unique.csv"))

In [27]:
zip_ll_file = os.path.join('Resources' , 'Zip_Lat_Long.txt')
zip_ll_df = pd.read_csv(zip_ll_file)
zip_ll_df.head()

,Zipcode,Lat,Lng
0,601,18.180555,-66.749961
1,602,18.361945,-67.175597
2,603,18.455183,-67.119887
3,606,18.158345,-66.932911
4,610,18.295366,-67.125135


In [28]:
rec_zip_df = pd.DataFrame(above50pct['Zipcode'].astype(int))
rec_locations_ll = rec_zip_df.merge(zip_ll_df, how='inner', on=['Zipcode'])

In [ ]:
#Plot Recommendations 
#Get lat, long of final recommendations
rec_locations_ll_df = rec_locations_ll[["Lat", "Lng"]].astype(float)

#Plot the lat long on Google map
fig = gmaps.figure()

try:
    starbucks_layer2 = gmaps.symbol_layer(rec_locations_ll_df, fill_color='red', stroke_color='red', scale=2)
except :
    print("Bad Lat Long")
    
fig.add_layer(starbucks_layer2)
fig

In [42]:
above10pct_zip_df = pd.DataFrame(above10pct['Zipcode'].astype(int))
above10pct_ll = above10pct_zip_df.merge(zip_ll_df, how='inner', on=['Zipcode'])

In [ ]:
#Plot Recommendations 
#Get lat, long of final recommendations
above10pct_ll_df = above10pct_ll[["Lat", "Lng"]].astype(float)

#Plot the lat long on Google map
fig3 = gmaps.figure()

try:
    starbucks_layer4 = gmaps.symbol_layer(above10pct_ll_df, fill_color='red', stroke_color='red', scale=1)
except :
    print("Bad Lat Long")
    
fig3.add_layer(starbucks_layer4)
fig3

In [37]:
project1_zip_df = pd.DataFrame(project1_df['Zipcode'].astype(int))
project1_locations_ll = project1_zip_df.merge(zip_ll_df, how='inner', on=['Zipcode'])

In [ ]:
#Plot Recommendations 
#Get lat, long of final recommendations
project1_locations_ll_df = project1_locations_ll[["Lat", "Lng"]].astype(float)

#Plot the lat long on Google map
fig2 = gmaps.figure()

try:
    starbucks_layer3 = gmaps.symbol_layer(project1_locations_ll_df, fill_color='blue', stroke_color='blue', scale=1)
except :
    print("Bad Lat Long")
    
fig2.add_layer(starbucks_layer3)
fig2